# Create perturbed FY files using CEA conservative (C1) evaluation

In [1]:
import sandy
from os.path import join, dirname
import pandas as pd
import numpy as np
import random, sys

## Extract FYs and covariance data for U235 thermal fission

In [3]:
path = join(dirname(sandy.__file__), 'appendix', 'fission_yields')

file_eval = r"mixt_cea-jeff33_u_235_th_eval_c1.stn"
file_cov = r"mixt_cea-jeff33_u_235_th_ind_cov_mat_c1"
tape = sandy.Endf6.from_file(join(path, file_eval))
nfpy = sandy.Fy.from_endf6(tape)

idx = nfpy.data.query(f"E==0.0253 & MT==454").index
ifyth = nfpy.data.loc[idx]

cov = pd.read_csv(join(path, file_cov), sep=r"\s+", header=None)
cov.index = cov.columns = pd.Index(ifyth.ZAP)
cov = sandy.CategoryCov(cov)

## Generate perturbation coefficients and write them to file

In [4]:
seed = random.randrange(2**32 - 1)    # create a seed
seed = 1444271359                     # or use always the same
nsmp = 10                             # sample size
smp = cov.sampling(nsmp, seed=seed)

In [5]:
with pd.ExcelWriter('PERT_92235TH_MF8_MT454.xlsx') as writer:
    pd.DataFrame([[seed]]).to_excel(writer, sheet_name='SEED', columns=None, index=False)
    smp.data.to_excel(writer, sheet_name='IFY_TH')

## Read coefficients from perturbation file and generate random FY ENDF-6 files

Skip the part above if you already have the file of perturbations.

In [6]:
smp = pd.read_excel("PERT_92235TH_MF8_MT454.xlsx", sheet_name="IFY_TH", index_col=0)
smp = sandy.Samples(smp)

In [7]:
path = join(dirname(sandy.__file__), 'appendix', 'fission_yields')

file_eval = r"mixt_cea-jeff33_u_235_th_eval_c1.stn"
tape = sandy.Endf6.from_file(join(path, file_eval))
nfpy = sandy.Fy.from_endf6(tape)
idx_ify = nfpy.data.query(f"E==0.0253 & MT==454").index
idx_cfy = nfpy.data.query(f"E==0.0253 & MT==459").index

In [ ]:
tape_rdd = sandy.get_endf6_file("jeff_33", "decay", "all")
rdd = sandy.DecayData.from_endf6(tape_rdd)  # this can take a while

In [ ]:
smp_min = 0   # write ENDF-6 file only in the sample range [smp_min, smp_max]
smp_max = 9
file_template = "u235_fy_cea_c1_{}.jeff33"
for ismp in range(smp_min, smp_max+1):
    file = file_template.format(ismp)
    f = sandy.Fy(nfpy.data.copy())
    f.data.loc[idx_ify, "DFY"] = f.data.loc[idx_ify, "FY"]            # just for me, i copy the original IFYs where uncertainties should be, so i can compare them to the perturbed ones (anyways I don't use uncertainties)
    f.data.loc[idx_cfy, "DFY"] = f.data.loc[idx_cfy, "FY"]            # same but for CFYs
    f.data.loc[idx_ify, "FY"] *= smp.data[ismp].values                # IMPORTANT, this does not update the CFYs, which in random ENDF-6 file are inconsistent with the perturbed IFYs
    f = f.apply_qmatrix(922350, 0.0253, rdd, keep_fy_index=True)      # Run this if you want to update the CFYs (slower), or else comment it out
    print(f"writing file '{file}'...")
    f.to_endf6(tape).to_file(file)